# Supervised Machine Learning Models - Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.3f}'.format

#### Two Types of Supervised Machine Learning Models

1. **Classification**: Models where we seek a yes-or-no prediction, such as "Is this financial transaction fraudulent?", "Will this firm default on its debt?", and so on.

2. **Continuous**: Models where the value being predicted falls somewhere on a continuous spectrum such as a predicted sales figure or a financial risk score.

#### Regression

Regression analysis (including linear and logistic regression), is a form of supervised machine learning. We will use regression analysis to fit a model to data within a training sample and then use that model to predict a dependent variable in an out-of-sample testing set.

Logisitic regression is an example of a classification model, while linear regression is an example of a continuous model.

In this lecture, we'll discuss logistic regression.

#### Diving Into Some Data

To illustrate some basic concepts, we'll be using the **default.csv** dataset (a synthetic dataset) which contains data on companies that have and have not declared bankruptcy from 2002 to 2014. Our goal will be to create a machine learning algorithm that effectively predicts the likelihood that a company will file for bankruptcy within one year.

| Variable | Definition |
| --- | --- |
| gvkey | Firm identifier |
| datadate | Fiscal period end date |
| default | An indicator variable equal to 1 if the firm defaults (i.e., declares bankruptcy) within one year |
| ln_mve | The natural logarithm of market value of equity |
| btm | book-to-market ratio (book value of equity divided by market value of equity |
| retvol | return volatility (standard deviation of monthly stock returns over the previous 5 years) |
| ch_price | one-year percentage change in stock price |
| roa | return on assets (income before extraordinary items / assets) |
| lev | leverage ratio (total liabilities / total assets) |
| std_income | the standard deviation of quarterly income before extraordinary items over the previous 5 years |

We'll include the following variables as indepedent variables in our logistic regression:

1. ln_mve
2. btm
3. retvol
4. ch_price
5. roa
6. lev
7. std_income

The dependent variable that we want to predict is **default**.

Let's now import the data.

In [2]:
df = pd.read_csv('default.csv')
df.head()

,gvkey,datadate,lev,roa,btm,retvol,ch_price,std_income,default,ln_mve
0,1,9/30/2011,1.196,-0.007,-4.873,0.133,-3.310,377.805,1,6.899
1,2,3/31/2002,0.371,-0.003,2.794,0.129,-11.947,853.407,1,9.902
2,3,12/31/2007,0.801,-0.024,1.603,0.173,-1.890,63.434,1,7.077
3,3,9/30/2008,0.869,-0.030,5.177,0.273,-11.050,99.032,1,5.407
4,4,6/30/2009,1.493,0.013,-2.452,0.516,-2.440,360.054,1,6.327


To estimate the logistic regression, we'll use the **statsmodels** module. Let's import it.

In [3]:
import statsmodels.api as sm

#### Prepare the training and testing sets

Now, let's create variables to capture the dependent and independent variables in our model in both the testing and training sets. We add a constant term using **assign(_const=1)**.

In [4]:
X = df[['ln_mve','btm','retvol','ch_price','roa','lev','std_income']].assign(_const=1)
y = df[['default']]

Next, let's split our data into a training set and a testing set. We'll randomly choose 80% of the data to use in our training set and the remaining 20% of the data to use in our testing set.

To do so, we can use the **train_test_split** function within the **sklearn.model_selection** module.

Options for **train_test_split**:
- **test_size**: the percentage of observations to include in the test set
- **random_state**: a 'seed' number which allows us to get the same random sample each time we run the code (which is useful for replication)
- **stratify=y**: require the mean of the dependent variable to be the same in the training and testing sets (only available for categorical dependent variables)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)

print(f'# Observations in X_train: {len(X_train)}')
print(f'# Observations in y_train: {len(y_train)}')
print(f'# Observations in X_test: {len(X_test)}')
print(f'# Observations in y_test: {len(y_test)}')

# Observations in X_train: 1267
# Observations in y_train: 1267
# Observations in X_test: 317
# Observations in y_test: 317


In [6]:
display(X_train.head())
display(y_train.head())

,ln_mve,btm,retvol,ch_price,roa,lev,std_income,_const
920,6.444,0.401,0.101,-7.090,-0.097,0.563,26.734,1
1211,8.492,0.548,0.052,22.730,0.017,0.658,382.123,1
1297,12.396,0.346,0.031,4.700,0.022,0.498,"1,306.063",1
1401,7.918,0.380,0.082,3.360,0.020,0.585,23.910,1
740,6.940,0.184,0.112,-4.590,0.009,0.565,2.417,1


,default
920,0
1211,0
1297,0
1401,0
740,0


In [7]:
display(y_train.describe())
display(y_test.describe())

,default
count,"1,267.000"
mean,0.091
std,0.287
min,0.000
25%,0.000
50%,0.000
75%,0.000
max,1.000


,default
count,317.000
mean,0.091
std,0.289
min,0.000
25%,0.000
50%,0.000
75%,0.000
max,1.000


#### Estimate the model

We can now use the training data to estimate the logistic model. 

In [8]:
model = sm.Logit(y_train, X_train).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.175680
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 1267
Model:                          Logit   Df Residuals:                     1259
Method:                           MLE   Df Model:                            7
Date:                Fri, 29 Nov 2024   Pseudo R-squ.:                  0.4227
Time:                        22:41:32   Log-Likelihood:                -222.59
converged:                       True   LL-Null:                       -385.56
Covariance Type:            nonrobust   LLR p-value:                 1.734e-66
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ln_mve        -0.6230      0.086     -7.215      0.000      -0.792      -0.454
btm            0.0302      0.

#### Generate predictions in the testing set

Now, let's apply this model to the out-of-sample testing set to create the out-of-sample prediction. If the predicted value is above 0.50, we'll set the prediction to 1, otherwise, we'll set the prediction to 0.

In [9]:
y_test['default_p']=((model.predict(X_test)>0.5).astype(int))
y_test.head()

,default,default_p
783,0,0
624,0,0
775,0,0
355,0,1
1333,0,0


To add this prediction back to the original dataset, we can merge on the index.

In [10]:
predictions = y_test[['default_p']]
df = pd.merge(df,predictions,left_index=True,right_index=True,how='left')

df_pred = df[pd.isnull(df.default_p) == False]
df_pred.head()

,gvkey,datadate,lev,roa,btm,retvol,ch_price,std_income,default,ln_mve,default_p
6,6,6/30/2009,0.893,-0.022,2.558,0.380,-5.530,46.792,1,3.214,1.000
13,13,9/30/2011,1.323,-0.042,-7.839,0.292,-3.420,284.212,1,5.350,1.000
16,16,10/31/2008,0.928,-0.102,1.173,0.256,-8.550,20.792,1,3.536,1.000
19,19,8/31/2010,1.269,-0.056,-4.771,0.207,-3.670,187.303,1,5.142,1.000
20,20,12/31/2003,0.752,-0.036,0.863,0.141,-0.720,7.781,1,4.433,0.000


#### Evaluate the Model

Now, let's identify the number of observations that our model predicted to be default/not default vs. the actual number of observations that were default/not default.

In [11]:
assessments = y_test.groupby(['default','default_p'])['default_p'].describe().reset_index()
assessments = assessments[['default','default_p','count']]
assessments

,default,default_p,count
0,0,0,279.000
1,0,1,9.000
2,1,0,9.000
3,1,1,20.000


We can label these classifications as follows:

        True Negative  (TN) : 279
        True Positive  (TP) : 20
        False Negative (FN) : 9
        False Positive (FP) : 9

Let's write some code to pull these numbers out of the **assessments** DataFrame.

In [12]:
TN = assessments.iloc[0,2]
FP = assessments.iloc[1,2]
FN = assessments.iloc[2,2]
TP = assessments.iloc[3,2]

print(f'True Negative : {TN}')
print(f'True Positive : {TP}')
print(f'False Negative: {FN}')
print(f'False Positive: {FP}')

True Negative : 279.0
True Positive : 20.0
False Negative: 9.0
False Positive: 9.0


There are three useful metrics to evaluate our model: accuracy, sensitivity, and specificity.

#### Accuracy

Number of correct assessments / Number of all assessments

Accuracy = (TN + TP)/(TN + TP + FN + FP)

Measures the proportion of **actual** positives and negatives that are correctly identified as positives or negatives.

#### Sensitivity

Number of true positive assessments / Number of all positive assessments

Sensitivity = TP/(TP + FN)

Measures the proportion of **actual** positives that are correctly identified as positives.

#### Specificity

Number of true negative assessments / Number of all negative assessments

Specificity = TN/(TN + FP)

Measures the proportion of **actual** negatives that are correctly identified as negatives.

In [13]:
Accuracy = (TN + TP)/(TN + TP + FN + FP)
Sensitivity = TP/(TP + FN)
Specificity = TN/(TN + FP)

print(f'Accuracy    : {Accuracy:.3f}')
print(f'Sensitivity : {Sensitivity:.3f}')
print(f'Specificity : {Specificity:.3f}')

Accuracy    : 0.943
Sensitivity : 0.690
Specificity : 0.969


Another way to evaluate the prediction is to estimate a logistic regression with **default** as the dependent variable and **default_p** as the sole regressor and view the model summary.

In [14]:
model = sm.Logit(y_test['default'], y_test[['default_p']].assign(_const=1)).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.183001
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  317
Model:                          Logit   Df Residuals:                      315
Method:                           MLE   Df Model:                            1
Date:                Fri, 29 Nov 2024   Pseudo R-squ.:                  0.4019
Time:                        22:41:32   Log-Likelihood:                -58.011
converged:                       True   LL-Null:                       -96.988
Covariance Type:            nonrobust   LLR p-value:                 1.055e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
default_p      4.2325      0.525      8.059      0.000       3.203       5.262
_const        -3.4340      0.

#### Sensitivity vs. Specificity

If you lended money to this firm, which one would you like to be higher when predicting default -- sensitivity or specificity? 

Personally, I'd much rather have a false positive (i.e., the model says the firm will default when in fact it will not default) than to 
have a false negative (i.e., the model says the firm will not default when in fact it will default). In other words, I'd want false negatives to be as close to 0 as possible. I'd rather have assurance that my investment is safe. If I can perfectly predict when actual default occurs, I'd be okay sacrificing some investments in firms that might not have otherwise defaulted but that my model predicted would default. <u>So I'd probably want the sensitivity to be really high</u> with this kind of model (i.e., if the firm will default, I want the model to always tell me that it will default).

#### statsmodels vs. scikit-learn (i.e., "sklearn")

There are several ways to estimate logistic regression in Python. The **statsmodels** module is great because of the nice fancy output that it generates.

However, for machine learning, the **sklearn** module provides some really useful built-in functions to make our life a lot easier if our goal is out-of-sample prediction. We can use **sklearn** to implement many different machine learning algorithms.

Here is a list of all the models that **sklearn** can perform:

**Supervised Learning:**

- Generalized Linear Models:
    - Linear Regression
    - Ridge Regression
    - Lasso Regression
    - Elastic Net
    - Logistic Regression
    - Poisson Regression
    - Tweedie Regression
- Linear and Quadratic Discriminant Analysis
- Kernel Ridge Regression
- Support Vector Machines:
    - Support Vector Classifier
    - Support Vector Regression
    - Nu-Support Vector Classifier
    - Nu-Support Vector Regression
    - One-Class SVM
- Nearest Neighbors:
    - K-Nearest Neighbors Classifier
    - K-Nearest Neighbors Regressor
    - Radius Neighbors Classifier
    - Radius Neighbors Regressor
- Gaussian Processes:
    - Gaussian Process Classifier
    - Gaussian Process Regressor
- Decision Trees:
    - Decision Tree Classifier
    - Decision Tree Regressor
    - Random Forest Classifier
    - Random Forest Regressor
    - Extra Trees Classifier
    - Extra Trees Regressor
    - Gradient Boosting Classifier
    - Gradient Boosting Regressor
    - AdaBoost Classifier
    - AdaBoost Regressor
    - HistGradientBoostingClassifier
    - HistGradientBoostingRegressor
- Neural Network Models:
    - Multi-layer Perceptron Classifier
    - Multi-layer Perceptron Regressor
    - Bernoulli Restricted Boltzmann Machine
    - Gaussian Mixture Model Classifier
    - Autoencoder
- Naive Bayes:
    - Gaussian Naive Bayes
    - Multinomial Naive Bayes
    - Complement Naive Bayes
    - Bernoulli Naive Bayes
- Ensemble Methods:
    - Voting Classifier
    - Voting Regressor
    - Bagging Classifier
    - Bagging Regressor
    - Random Patches Classifier
    - Random Patches Regressor
    - Random Subspaces Classifier
    - Random Subspaces Regressor
    - Stacking Classifier
    - Stacking Regressor
    - ComplementNB
    - GradientBoostingClassifier

**Unsupervised Learning:**

- Clustering:
    - K-Means Clustering
    - Mini-Batch K-Means Clustering
    - Affinity Propagation
    - Mean Shift
    - Spectral Clustering
    - Agglomerative Clustering
    - DBSCAN
    - Birch
- Decomposition:
    - Principal Component Analysis (PCA)
    - Incremental PCA
    - Kernel PCA
    - Sparse PCA
    - Truncated SVD
    - Dictionary Learning
    - Independent Component Analysis (ICA)
    - Non-negative Matrix Factorization (NMF)
- Density Estimation:
    - Gaussian Mixture Models
    - Kernel Density Estimation
- Outlier Detection:
    - Elliptic Envelope
    - Isolation Forest
    - Local Outlier Factor

**Semi-Supervised Learning:**
- Label Propagation
- Label Spreading
- Self-Training
- Multi-View Learning
- MixMatch

For example, to implement the exact same logistic regression that we just computed, we use the **LogisticRegression** fuction within the **sklearn** module as follows:

In [15]:
from sklearn.linear_model import LogisticRegression

# Estimate the logistic regression within the training set

model = LogisticRegression(C=1e9, max_iter=10000).fit(X_train, y_train.values.ravel())
#       Note: The C=1e9 and max_iter=10000 make the sklearn version of logistic regression more like the statsmodels version. These parameters are optional.

# Apply the model to the out-of-sample testing set to create the out-of-sample prediction
# Note: The predict function for the sklearn module returns a 0/1 prediction if the predicted probability is > 50%
#       unlike the predict statement for the statsmodels module which returns a continuous prediction between 0 and 1.

y_test['default_p_alt'] = model.predict(X_test)
y_test.head()

,default,default_p,default_p_alt
783,0,0,0
624,0,0,0
775,0,0,0
355,0,1,1
1333,0,0,0


The **sklearn** module has a nice function called **confusion_matrix** which computes the number of True Negatives, False Positives, False Negatives, and True Positives (these are all contained in what we call a "confusion matrix"). We can then use these values to calculate accuracy, sensitivity, and specificity.

In [16]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test['default'], y_test['default_p_alt']) 
print(conf_mat)

TN, FP, FN, TP = confusion_matrix(y_test['default'], y_test['default_p_alt']).ravel() 

# Note: .ravel() simply unpacks the matrix into its four components (TN, FP, FN, TP)
  
Accuracy = (TN + TP)/(TN + TP + FN + FP)
Sensitivity = TP/(TP + FN)
Specificity = TN/(TN + FP)

print(f'Accuracy    : {Accuracy:.3f}')
print(f'Sensitivity : {Sensitivity:.3f}')
print(f'Specificity : {Specificity:.3f}')

model = sm.Logit(y_test['default'], y_test[['default_p_alt']].assign(_const=1)).fit()
print(model.summary())

[[279   9]
 [  9  20]]
Accuracy    : 0.943
Sensitivity : 0.690
Specificity : 0.969
Optimization terminated successfully.
         Current function value: 0.183001
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  317
Model:                          Logit   Df Residuals:                      315
Method:                           MLE   Df Model:                            1
Date:                Fri, 29 Nov 2024   Pseudo R-squ.:                  0.4019
Time:                        22:41:32   Log-Likelihood:                -58.011
converged:                       True   LL-Null:                       -96.988
Covariance Type:            nonrobust   LLR p-value:                 1.055e-18
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
default_p_alt     4

#### Exercise

In this exercise you will use the **WA_Fn-UseC_-HR-Employee-Attrition.csv** dataset to predict employee attrition. The data was obtained from https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset and contains the following variables:

| Variable | Definition |
| --- | --- |
| Attrition | 'Yes' if the employee leaves the company, 'No' if the employee stays with the company |
| EmployeeNumber | Unique identifier for each employee |
| Age | Age in years of the employee |
| BusinessTravel | Frequency of business travel: 'Frequently', 'Rarely' or 'Non-Travel' |
| DailyRate | Daily rate of pay for the employee |
| Department | Department the employee belongs to: 'Sales', 'Research & Development' or 'Human Resources' |
| DistanceFromHome | Distance from employee's home to workplace |
| Education | Level of education: 1 'Below College', 2 'College', 3 'Bachelor', 4 'Master', 5 'Doctor' |
| EducationField | Field of study in which the employee obtained their highest education: 'Life Sciences', 'Medical', 'Marketing', 'Technical Degree', 'Human Resources', 'Engineering', 'Arts', or 'Other' |
| EmployeeCount | Number of employees in the company |
| EnvironmentSatisfaction | Employee's level of satisfaction with their work environment: 1 'Low', 2 'Medium', 3 'High', 4 'Very High' |
| Gender | Employee's gender: 'Male' or 'Female' |
| HourlyRate | Hourly rate of pay for the employee |
| JobInvolvement | Employee's level of job involvement: 1 'Low', 2 'Medium', 3 'High', 4 'Very High' |
| JobLevel | Employee's job level: 1 'Entry Level', 2 'Intermediate Level', 3 'Managerial Level', 4 'Director Level', 5 'Executive Level' |
| JobRole | Employee's job role: 'Sales Executive', 'Research Scientist', 'Laboratory Technician', 'Manufacturing Director', 'Healthcare Representative', 'Manager', 'Sales Representative', 'Research Director', 'Human Resources' |
| JobSatisfaction | Employee's level of job satisfaction: 1 'Low', 2 'Medium', 3 'High', 4 'Very High' |
| MaritalStatus | Employee's marital status: 'Single', 'Married' or 'Divorced' |
| MonthlyIncome | Monthly income of the employee |
| MonthlyRate | Monthly rate of pay for the employee |
| NumCompaniesWorked | Number of companies the employee has worked for |
| Over18 | Whether the employee is over 18 years old: 'Y' or 'N' |
| OverTime | Whether the employee works overtime: 'Yes' or 'No' |
| PercentSalaryHike | Percentage increase in salary for the employee |
| PerformanceRating | Employee's performance rating: 1 'Low', 2 'Good', 3 'Excellent', 4 'Outstanding' |
| RelationshipSatisfaction | Employee's level of satisfaction with their relationships at work: 1 'Low', 2 'Medium', 3 'High', 4 'Very High' |
| StandardHours | Standard number of working hours for the company |
| StockOptionLevel | Employee's level of stock options: 0 'None', 1 'Low', 2 'Medium', 3 'High' |
| TotalWorkingYears | Total number of years the employee has worked |
| TrainingTimesLastYear | Number of times the employee received training last year |
| WorkLifeBalance | Employee's level of work-life balance: 1 'Bad', 2 'Good', 3 'Better', 4 'Best' |
| YearsAtCompany | Number of years the employee has worked at the company |
| YearsInCurrentRole | Number of years the employee has been in their current role |
| YearsSinceLastPromotion | Number of years since the employee's last promotion |
| YearsWithCurrManager | Number of years the employee has been working under their current manager |

Use logistic regression to predict whether the employee will leave the company (i.e., **Attrition**) using all possible independent variables in the model. Train the model on a random sample of 75% of the observations in the dataset, and test the model on the remaining 25% of the observations in the dataset. How well did the model perform?